In [ ]:
import fsspec
import geopandas as gpd
import hvplot.xarray
import pystac
import rioxarray
import shapely
import xarray as xr
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

In [ ]:
m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = 46.34, -1.47
m.zoom = 15
m.layout.height = "800px"
m

In [ ]:
with fsspec.open(
    "https://coclico.blob.core.windows.net/tiles/S2A_OPER_GIP_TILPAR_MPC.parquet", "rb"
) as f:
    s2grid = gpd.read_parquet(f).to_crs(4326)

In [ ]:
import os

import dotenv

dotenv.load_dotenv()
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}

In [ ]:
import dask_geopandas as dgpd
import fsspec
import geopandas as gpd

from coastpy.geo.quadtiles import make_mercantiles

grid = make_mercantiles(5)

# Load coastline buffer data (2km buffer around the coastline) and convert to WGS84
buffer = dask_geopandas.read_parquet(
    "az://coastline-buffer/osm-coastlines-buffer-2000m.parquet",
    storage_options=storage_options,
).compute()

In [ ]:
"""
Script to generate a processing grid for scalable geospatial analysis.
This script:
1. Generates a Mercator grid based on a specified zoom level.
2. Reads and processes coastline buffer data.
3. Clips the buffer to the European region.
4. Filters tiles based on intersection with the coastline.
5. # TODO: Clips tiles based on a coastline buffer.  

Dependencies: dask-geopandas, geopandas, fsspec
"""

import dask_geopandas as dgpd
import fsspec
import geopandas as gpd

from coastpy.geo.quadtiles import make_mercantiles

grid = make_mercantiles(5)

# Load coastline buffer data (2km buffer around the coastline) and convert to WGS84
buffer = dask_geopandas.read_parquet(
    "az://coastline-buffer/osm-coastlines-buffer-2000m.parquet",
    storage_options=storage_options,
).compute()


# Load coastline data
coastline_urlpath = "az://coastlines-osm/release/2023-02-09/coast_3857_gen9.parquet"
with fsspec.open(coastline_urlpath, mode="rb", **storage_options) as f:
    coastline = gpd.read_parquet(f).to_crs(4326)


# Load countries dataset and filter for Europe
with fsspec.open(
    "https://coclico.blob.core.windows.net/public/countries.parquet", "rb"
) as f:
    countries = gpd.read_parquet(f)

europe = countries[
    (countries["continent"] == "EU")
    & (~countries["common_country_name"].isin(["Svalbard", "Russia"]))
]

# Clip the coastline buffer to the European region
european_coastline = gpd.clip(coastline, europe)

# Filter the Mercator grid tiles by spatial join with the European buffer
filtered_tiles = grid.sjoin(european_coastline, how="inner")

# Drop unnecessary columns to clean the output
filtered_tiles = filtered_tiles.drop(columns=["index_right"])

# TODO: I want the tiles to be clipped by the buffer. So that within each tile
# I only have the buffer area (coastal zone)
...  # < your code comes here.
clipped_tiles = gpd.overlay(filtered_tiles, european_buffer, how="intersection")

In [ ]:
def buffer_antimeridian(buffer_size):
    """
    Create a buffer around the antimeridian to handle crossings.

    Args:
        buffer_size (float): Buffer distance in degrees.

    Returns:
        gpd.GeoDataFrame: Buffered antimeridian area.
    """
    antimeridian = shapely.geometry.LineString([(180, -90), (180, 90)])
    buffer_geom = shapely.geometry.Polygon(antimeridian.buffer(buffer_size))
    return gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:4326")


buffer_antimeridian()

In [ ]:
geom = gpd.GeoDataFrame(
    geometry=[shapely.geometry.LineString([(180, -90), (180, 90)])], crs=4326
)


utm_northeast = 32660
utm_northwest = 32601
utm_southeast = ...
utm_southwest = ...

print(list(geom.to_crs(utm_crs).geometry.item().coords))

In [ ]:
import geopandas as gpd
import shapely.geometry
from pyproj import CRS, Transformer
from shapely.ops import transform


def create_antimeridian_buffer(buffer_size_meters):
    """
    Create a valid buffer area around the antimeridian.

    Args:
        buffer_size_meters (float): Buffer distance in meters.

    Returns:
        gpd.GeoDataFrame: Buffered area around the antimeridian in EPSG:4326.
    """
    # Step 1: Define the antimeridian line
    antimeridian_line = gpd.GeoDataFrame(
        geometry=[shapely.geometry.LineString([(180, -90), (180, 90)])], crs="EPSG:4326"
    )

    # Step 2: Define UTM zones for each quadrant
    utm_zones = {
        "northeast": 32660,  # UTM zone 60N
        "northwest": 32601,  # UTM zone 1N
        "southeast": 32760,  # UTM zone 60S
        "southwest": 32701,  # UTM zone 1S
    }

    buffered_quadrants = []

    # Step 3 & 4: Transform to UTM, compute buffer, and convert back
    for quadrant, utm_crs in utm_zones.items():
        transformer_to_utm = Transformer.from_crs(
            "EPSG:4326", CRS.from_epsg(utm_crs), always_xy=True
        )
        transformer_to_4326 = Transformer.from_crs(
            CRS.from_epsg(utm_crs), "EPSG:4326", always_xy=True
        )

        # Transform the antimeridian line to UTM CRS
        utm_line = transform(
            transformer_to_utm.transform, antimeridian_line.geometry.item()
        )

        # Buffer in UTM CRS
        utm_buffer = shapely.geometry.Polygon(
            [
                (utm_line.coords[0][0] - buffer_size_meters, utm_line.coords[0][1]),
                (utm_line.coords[1][0] - buffer_size_meters, utm_line.coords[1][1]),
                (utm_line.coords[1][0] + buffer_size_meters, utm_line.coords[1][1]),
                (utm_line.coords[0][0] + buffer_size_meters, utm_line.coords[0][1]),
            ]
        )

        # Convert buffered geometry back to EPSG:4326
        epsg4326_buffer = transform(transformer_to_4326.transform, utm_buffer)

        # Append to the list of buffered quadrants
        buffered_quadrants.append(epsg4326_buffer)

    # Step 5: Combine buffered quadrants into one GeoDataFrame
    combined_buffers = gpd.GeoDataFrame(geometry=buffered_quadrants, crs="EPSG:4326")

    # Step 6: Validate geometries
    combined_buffers["is_valid"] = combined_buffers.geometry.is_valid
    if not combined_buffers["is_valid"].all():
        print("Some geometries are invalid. Attempting to fix...")
        combined_buffers["geometry"] = combined_buffers.geometry.apply(
            lambda geom: geom.buffer(0) if not geom.is_valid else geom
        )

    return combined_buffers


# Example Usage
buffer_size = 300000  # 300 km buffer
antimeridian_buffer = create_antimeridian_buffer(buffer_size)
print(antimeridian_buffer)

In [ ]:
from geopy.distance import geodesic


def compute_geodesic_point(lat, lon, distance_km, bearing):
    """
    Compute a geodesic destination point.

    Args:
        lat (float): Latitude of the starting point.
        lon (float): Longitude of the starting point.
        distance_km (float): Distance to travel in kilometers.
        bearing (float): Direction of travel in degrees (e.g., 90 for east).

    Returns:
        tuple: (latitude, longitude) of the destination point.
    """
    start_point = (lat, lon)
    destination = geodesic(kilometers=distance_km).destination(start_point, bearing)
    return destination.latitude, destination.longitude


# Example Usage:
start_lat = 0
start_lon = -180
distance = 5  # in kilometers
bearing = 90  # eastward

destination_point = compute_geodesic_point(start_lat, start_lon, distance, bearing)
print(f"Destination Point: {destination_point}")

In [ ]:
start_lat, start_lon = 0.1, -180  # Antimeridian equator point
shifted_lat, shifted_lon = compute_geodesic_point(start_lat, start_lon, 5, 90)

p1 = shapely.Point(start_lon, start_lat)
p2 = shapely.Point(shifted_lon, shifted_lat)

gdf1 = gpd.GeoDataFrame(geometry=[p1], crs=4326)
gdf2 = gpd.GeoDataFrame(geometry=[p2], crs=4326)

m = gdf1.explore()
gdf2.explore(m=m, color="red")

utm_crs = gdf2.estimate_utm_crs()

line = gpd.GeoDataFrame(
    geometry=[shapely.LineString([shapely.Point(p2.x, 0), shapely.Point(p2.x, 84)])],
    crs=4326,
)
line.assign(geometry=line.to_crs(utm_crs).buffer(2500)).to_file(
    "/Users/calkoen/tmp/test.gpkg"
)

In [ ]:
import antimeridian
from shapely.geometry import shape
from shapely.validation import make_valid


def correct_antimeridian_cross(row):
    """
    Correct geometries crossing the antimeridian using the antimeridian library.

    Args:
        row (pd.Series): Row containing the geometry to correct.

    Returns:
        shapely.geometry.base.BaseGeometry: Corrected geometry.
    """
    geom = row.geometry

    try:
        # Convert GeoJSON-like geometries to Shapely if necessary
        if isinstance(geom, dict):
            geom = shape(geom)

        # Ensure geometry is valid
        if not geom.is_valid:
            geom = make_valid(geom)

        # Fix geometry using antimeridian library
        return antimeridian.fix_polygon(geom, fix_winding=False)
    except Exception as e:
        # Log and return the original geometry if correction fails
        print(e)
        return geom


def correct_antimeridian_crosses_in_df(df):
    """
    Correct geometries that cross the antimeridian.

    Args:
        df (gpd.GeoDataFrame): Input GeoDataFrame with `crosses_antimeridian` column.
        utm_grid (gpd.GeoDataFrame): UTM grid for overlay.

    Returns:
        gpd.GeoDataFrame: Updated GeoDataFrame with corrected geometries.
    """
    df = df.copy()

    # Create a boolean mask for rows to correct
    rows_to_correct = df["crosses_antimeridian"]

    # Apply the correction only to rows where `crosses_antimeridian` is True
    df.loc[rows_to_correct, "geometry"] = df.loc[rows_to_correct].apply(
        lambda row: correct_antimeridian_cross(row), axis=1
    )
    return df


gdf = gpd.read_parquet("/Users/calkoen/tmp/data.parquet")

gdf2 = correct_antimeridian_crosses_in_df(gdf)

In [ ]:
buf = gpd.read_file("/Users/calkoen/data/prc/test_buffer_15000_coast_3857_gen9.gpkg")

In [ ]:
df = buf.copy()
df[(df.geom_type == "Polygon") | (df.geom_type == "MultiPolygon")]

In [ ]:
buf.shape

In [ ]:
buf[buf.geom_type == "LineString"].explore()

In [ ]:
def crosses_antimeridian(geometry):
    """
    Check if a geometry crosses the antimeridian.

    Args:
        geometry (shapely.geometry.base.BaseGeometry): The input geometry.

    Returns:
        bool: True if the geometry crosses the antimeridian, False otherwise.
    """
    minx, miny, maxx, maxy = geometry.bounds
    return maxx - minx > 180


def map_crosses_antimeridian(df):
    src_crs = df.crs
    df = df.to_crs(4326)
    df["crosses_antimeridian"] = df["geometry"].apply(crosses_antimeridian)
    df = df.to_crs(src_crs)
    return df


r = map_crosses_antimeridian(buf)

In [ ]:
import antimeridian
from shapely.validation import make_valid

def correct_antimeridian_cross(row):
    """
    Correct geometries crossing the antimeridian using the antimeridian library.

    Args:
        row (pd.Series): Row containing the geometry to correct.

    Returns:
        shapely.geometry.base.BaseGeometry: Corrected geometry.
    """
    geom = row.geometry

    try:
        # Fix geometry using antimeridian library
        import antimeridian

        if geom.geom_type == "Polygon":
            fixed = antimeridian.fix_polygon(geom, fix_winding=True)
            fixed = make_valid(fixed)
            return fixed
        elif geom.geom_type == "MultiPolygon":
            fixed = antimeridian.fix_multi_polygon(geom, fix_winding=True)
            fixed = make_valid(fixed)
            return fixed

    except Exception as e:
        print(e)
        return None


def correct_antimeridian_crosses_in_df(df):
    """
    Correct geometries that cross the antimeridian.

    Args:
        df (gpd.GeoDataFrame): Input GeoDataFrame with `crosses_antimeridian` column.
        utm_grid (gpd.GeoDataFrame): UTM grid for overlay.

    Returns:
        gpd.GeoDataFrame: Updated GeoDataFrame with corrected geometries.
    """
    df = df.copy()
    crs = df.crs
    df = df.to_crs(4326)

    # Create a boolean mask for rows to correct
    rows_to_correct = df["crosses_antimeridian"]

    # Apply the correction only to rows where `crosses_antimeridian` is True
    df.loc[rows_to_correct, "geometry"] = df.loc[rows_to_correct].apply(
        lambda row: correct_antimeridian_cross(row), axis=1
    )
    df = df.to_crs(crs)
    return df

r2 = correct_antimeridian_crosses_in_df(r)

In [ ]:
r3 = r2[r2["crosses_antimeridian"]]

In [ ]:
r3.explore()

In [ ]:
buf.iloc[[299]].explore()

In [ ]:
m = gdf.explore()
gdf2.explore(m=m, color="red")

In [ ]:
)
print(gdf.geom_type.unique())

In [ ]:
m = gdf2.explore()
gdf.explore(m=m, color="red")

In [ ]:
gdf

In [ ]:
import geopandas as gpd
from geopy.distance import geodesic
from shapely.geometry import LineString


def compute_geodesic_point(lat, lon, distance_km, bearing):
    start_point = (lat, lon)
    destination = geodesic(kilometers=distance_km).destination(start_point, bearing)
    return destination.latitude, destination.longitude


def create_antimeridian_buffers(buffer_size_km=5):
    # Define UTM zones
    utm_zones = {
        "northeast": 32660,  # UTM zone 60N
        "northwest": 32601,  # UTM zone 1N
        "southeast": 32760,  # UTM zone 60S
        "southwest": 32701,  # UTM zone 1S
    }

    # Initialize storage for polygons
    buffered_geometries = []

    # Process each quadrant
    for quadrant, utm_zone in utm_zones.items():
        # Compute east/west starting point
        direction = 90 if "east" in quadrant else 270  # East for NE/SE, West for NW/SW
        start_lat, start_lon = 0, -180  # Antimeridian equator point
        shifted_lat, shifted_lon = compute_geodesic_point(
            start_lat, start_lon, 5, direction
        )

        # Create north-south linestring
        if "north" in quadrant:
            line_coords = [(shifted_lon, 0), (shifted_lon, 90)]  # Northward
        else:
            line_coords = [(shifted_lon, 0), (shifted_lon, -90)]  # Southward

        line = LineString(line_coords)

        # Convert to UTM, buffer, and back to EPSG:4326
        gdf = gpd.GeoDataFrame(geometry=[line], crs="EPSG:4326").to_crs(utm_zone)
        gdf["geometry"] = gdf.buffer(buffer_size_km * 1000)  # Buffer in meters
        gdf = gdf.to_crs("EPSG:4326")

        # Store result with metadata
        gdf["quadrant"] = quadrant
        buffered_geometries.append(gdf)

    # Combine all buffered geometries into a single GeoDataFrame
    result_gdf = gpd.GeoDataFrame(pd.concat(buffered_geometries, ignore_index=True))

    # Validate and fix geometries
    result_gdf = result_gdf.set_geometry("geometry")
    if not result_gdf.is_valid.all():
        result_gdf["geometry"] = result_gdf["geometry"].apply(
            lambda geom: geom.buffer(0)
        )

    return result_gdf


# Run the function
buffered_antimeridian = create_antimeridian_buffers(buffer_size_km=5)

# Visualize the result
buffered_antimeridian.explore(tooltip="quadrant")

In [ ]:
import geopandas as gpd
import pandas as pd
from geopy.distance import geodesic
from shapely.geometry import Point


def compute_geodesic_point(lat, lon, distance_km, bearing):
    """
    Compute a geodesic destination point.

    Args:
        lat (float): Latitude of the starting point.
        lon (float): Longitude of the starting point.
        distance_km (float): Distance to travel in kilometers.
        bearing (float): Direction of travel in degrees (e.g., 90 for east).

    Returns:
        tuple: (latitude, longitude) of the destination point.
    """
    start_point = (lat, lon)
    destination = geodesic(kilometers=distance_km).destination(start_point, bearing)
    return destination.latitude, destination.longitude


# Compute multiple geodesic points
start_lat = 0
start_lon = -180
distances_km = [5, 10, 15, 20, 25]  # Distances in km
bearing = 90  # Eastward

# Generate destination points
points = [
    compute_geodesic_point(start_lat, start_lon, dist, bearing) for dist in distances_km
]

# Create a DataFrame and GeoDataFrame
df = pd.DataFrame(points, columns=["latitude", "longitude"])
df["geometry"] = [Point(lon, lat) for lat, lon in zip(df["latitude"], df["longitude"])]

gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Visualize on a map
gdf.explore(
    tooltip=["latitude", "longitude"],
    popup=True,
    color="blue",
    marker_kwds={"radius": 5},
    title="Geodesic Points East of Antimeridian",
)

In [ ]:
from geopy.distance import geodesic


def compute_geodesic_point(start_point, distance_km, bearing_degrees):
    """
    Compute a geodesic point from a starting coordinate traveling a specified
    distance in a specified direction.

    Args:
        start_point (tuple): The starting coordinate (latitude, longitude) as a tuple.
        distance_km (float): The distance to travel in kilometers.
        bearing_degrees (float): The bearing in degrees (e.g., 90 for east, 270 for west).

    Returns:
        tuple: The destination coordinate (latitude, longitude).
    """
    # Use geopy to calculate the destination point
    destination = geodesic(kilometers=distance_km).destination(
        start_point, bearing_degrees
    )
    return destination.latitude, destination.longitude


# Example usage:
start_coord = (180, 0)  # Antimeridian, equator
distance = 5  # Kilometers
bearing = 90  # East

new_coord = compute_geodesic_point(start_coord, distance, bearing)

In [ ]:
p1 = shapely.Point(start_coord)
p2 = shapely.Point(new_coord)
gpd.GeoDataFrame(geometry=[p1, p2], crs=4326).explore()

In [ ]:
geom.explore()

In [ ]:
import geopandas as gpd
import shapely.geometry
from pyproj import CRS, Transformer


def buffer_antimeridian(buffer_size_meters):
    """
    Create a valid buffer around the antimeridian to handle crossings.

    Args:
        buffer_size_meters (float): Buffer size in meters.

    Returns:
        gpd.GeoDataFrame: Buffered antimeridian area as valid polygons.
    """
    # Define UTM zones for east and west of the antimeridian
    utm_zone_east = 60  # UTM zone just east of the antimeridian
    utm_zone_west = 1  # UTM zone just west of the antimeridian

    # Define UTM CRS for each zone
    utm_crs_east = CRS.from_epsg(32660)  # Northern hemisphere, UTM zone 60
    utm_crs_west = CRS.from_epsg(32601)  # Northern hemisphere, UTM zone 1

    # Define the antimeridian in latitude bounds
    lat_min, lat_max = -90, 90

    # Create transformers to convert EPSG:4326 to UTM and back
    to_utm_east = Transformer.from_crs("EPSG:4326", utm_crs_east, always_xy=True)
    to_utm_west = Transformer.from_crs("EPSG:4326", utm_crs_west, always_xy=True)
    to_epsg4326 = Transformer.from_crs(utm_crs_east, "EPSG:4326", always_xy=True)

    # Define east buffer polygon in UTM coordinates
    east_poly_coords = [
        to_utm_east.transform(180, lat_min),  # Start at (180°, lat_min)
        to_utm_east.transform(180 + buffer_size_meters / 111320, lat_min),  # East edge
        to_utm_east.transform(180 + buffer_size_meters / 111320, lat_max),  # East edge
        to_utm_east.transform(180, lat_max),  # Back at (180°, lat_max)
        to_utm_east.transform(180, lat_min),  # Close the polygon
    ]
    east_polygon = shapely.geometry.Polygon(east_poly_coords)

    # Define west buffer polygon in UTM coordinates
    west_poly_coords = [
        to_utm_west.transform(-180, lat_min),  # Start at (-180°, lat_min)
        to_utm_west.transform(-180 - buffer_size_meters / 111320, lat_min),  # West edge
        to_utm_west.transform(-180 - buffer_size_meters / 111320, lat_max),  # West edge
        to_utm_west.transform(-180, lat_max),  # Back at (-180°, lat_max)
        to_utm_west.transform(-180, lat_min),  # Close the polygon
    ]
    west_polygon = shapely.geometry.Polygon(west_poly_coords)

    # Convert polygons back to EPSG:4326
    east_polygon_epsg4326 = shapely.ops.transform(to_epsg4326.transform, east_polygon)
    west_polygon_epsg4326 = shapely.ops.transform(to_epsg4326.transform, west_polygon)

    # Combine both polygons into a GeoDataFrame
    combined_polygons = gpd.GeoDataFrame(
        geometry=[east_polygon_epsg4326, west_polygon_epsg4326], crs="EPSG:4326"
    )

    return combined_polygons


# Example usage
buffer_size = 300000  # Buffer size in meters
antimeridian_buffer = buffer_antimeridian(buffer_size)
print(antimeridian_buffer)

In [ ]:
antimeridian_buffer.iloc[[1]].explore()

In [ ]:
filtered_tiles

In [ ]:
r.explore()

In [ ]:
filtered_tiles.sample(1).explore()

In [ ]:
import antimeridian
import geopandas as gpd
from shapely.geometry import mapping, shape


def fix_geom(geom):
    """
    Fix geometries using the antimeridian library.

    Args:
        geom (shapely.geometry.base.BaseGeometry): Input geometry.

    Returns:
        shapely.geometry.base.BaseGeometry: Fixed geometry or the original geometry if it can't be fixed.
    """
    try:
        # Fix geometry using antimeridian library
        if geom.is_empty:
            return geom  # Skip empty geometries
        elif geom.is_valid:
            return geom  # Return if already valid

        # Try fixing specific geometry types
        if geom.geom_type == "Polygon":
            return antimeridian.fix_polygon(geom)
        elif geom.geom_type == "MultiPolygon":
            return antimeridian.fix_multipolygon(geom)
        else:
            # Attempt general fix for other shapes
            return antimeridian.fix_shape(geom)
    except Exception as e:
        # Log an issue with fixing and return the original geometry
        print(f"Could not fix geometry: {e}")
        return geom


def fix_invalid_geometries(gdf):
    """
    Fix invalid geometries in a GeoDataFrame.

    Args:
        gdf (geopandas.GeoDataFrame): Input GeoDataFrame.

    Returns:
        geopandas.GeoDataFrame: GeoDataFrame with fixed geometries.
    """
    # Identify invalid geometries
    invalid_mask = ~gdf.is_valid
    invalid_geometries = gdf.loc[invalid_mask]

    print(f"Found {len(invalid_geometries)} invalid geometries.")

    # Apply the fix_geom function to invalid geometries
    gdf.loc[invalid_mask, "geometry"] = invalid_geometries.geometry.map(fix_geom)

    # Re-check if geometries are now valid
    still_invalid = gdf[~gdf.is_valid]
    if len(still_invalid) > 0:
        print(f"Still invalid geometries: {len(still_invalid)}")
    else:
        print("All geometries are now valid.")

    return gdf


buffer = fix_invalid_geometries(coastline_buffer)
buffer = buffer[buffer.is_valid]

In [ ]:
from coastpy.io.utils import name_data

name_data(
    buffer, include_random_hex=False, filename_prefix="osm_coastline_buffer_2000m"
)

In [ ]:
tolerance = 0.01  # Adjust this value based on your acceptable resolution loss
simplified_buffer = buffer.copy()
simplified_buffer["geometry"] = buffer.geometry.simplify(
    tolerance, preserve_topology=True
)

# Save the simplified buffer
simplified_buffer.to_parquet("/Users/calkoen/tmp/buffer_simplified.parquet")

In [ ]:
# simplified_buffer.to_file("/Users/calkoen/tmp/buffer_simplified.gpkg")
buffer.to_file("/Users/calkoen/tmp/buffer.gpkg")

In [ ]:
invals = buffer[~buffer.is_valid]

In [ ]:
buffer.to_parquet("/Users/calkoen/tmp/buffer.parquet")

In [ ]:
invals.explore()

In [ ]:
import antimeridian
import shapely


def fix_geom(geom):
    try:
        if geom.dtype == 
        geom = antimeridian.fix_shape(geom)
    except:
        print("not fixedj")
        return geom
        

invalids.geometry.map(fix_geom)

In [ ]:
invalids.explore()

In [ ]:
import geodatasets

geodatasets.data

In [ ]:
gpd.read_file(gpd.datasets.get_path("natearth_lowres"))

In [ ]:
# NOTE: LOAD DATA
s2_tiles = retrieve_s2_tiles().to_crs(4326)
rois = retrieve_rois().to_crs(4326)
# TODO: write STAC catalog for the coastal buffer
buffer = dask_geopandas.read_parquet(
    "az://coastline-buffer/osm-coastlines-buffer-2000m.parquet",
    storage_options=storage_options,
).compute()
quadtiles = make_mercantiles(zoom_level=MERCANTILES_ZOOM_LEVEL).to_crs(4326)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    classifier = retrieve_coastsat_classifier()

region_of_interest = infer_region_of_interest(ROI)

# NOTE: make an overlay with a coastline buffer to avoid querying data we do not need
buffer_aoi = gpd.overlay(buffer, region_of_interest[["geometry"]].to_crs(buffer.crs))

# TODO: add heuristic to decide which s2 tiles to use
s2_tilenames_to_process = gpd.sjoin(
    s2_tiles, buffer_aoi.to_crs(s2_tiles.crs)
).Name.unique()

In [ ]:
west, south, east, north = m.west, m.south, m.east, m.north
# Note: small little hack to ensure the notebook also works when running all cells at once
if not west:
    west, south, east, north = (
        -1.4987754821777346,
        46.328320550966765,
        -1.446976661682129,
        46.352022707044455,
    )
roi = gpd.GeoDataFrame(
    geometry=[shapely.geometry.box(west, south, east, north)], crs=4326
)

In [ ]:
import planetary_computer as pc

from coastpy.eo.collection import (
    CopernicusDEMCollection,
    DeltaDTMCollection,
    S2Collection,
)

s2_ = (
    S2Collection()
    .search(
        roi,
        datetime_range="2023-05-01/2023-07-31",
        query={"eo:cloud_cover": {"lt": 10}},
    )
    .load(
        bands=["blue", "green", "red", "nir", "swir16"],
        # composite=50,
        spectral_indices=["NDWI", "NDVI"],
        chunks={"x": 256, "y": 256},
        patch_url=pc.sign,
    )
    .execute()
)

deltadtm = DeltaDTMCollection().search(roi).load().execute()
cop_dem = CopernicusDEMCollection().search(roi).load(patch_url=pc.sign).execute()

In [ ]:
s2 = s2_.compute()

In [ ]:
def median_composite(ds):
    """
    Compute the median composite along the 'time' dimension.

    Args:
        ds (xr.Dataset): Input dataset with dimensions (time, y, x).

    Returns:
        xr.Dataset: Median composite with dimensions (y, x).
    """
    composite = xr.apply_ufunc(
        np.nanmedian,
        ds,  # Input dataset
        input_core_dims=[["time"]],  # Operate along "time" dimension
        output_core_dims=[[]],  # Collapsed "time" dimension
        vectorize=True,  # Vectorize to apply along (y, x)
        dask="parallelized",  # Support for Dask arrays
        output_dtypes=[np.float32],  # Match input dtype
    )
    return composite


composite = median_composite(s2)

In [ ]:
c2 = s2.median("time", skipna=True, keep_attrs=True)

In [ ]:
s2.quantile(0.15, dim="time", skipna=True, keep_attrs=True)

In [ ]:
import time

import numpy as np
import pandas as pd
import xarray as xr


def benchmark(func, *args, **kwargs):
    """
    Helper function to benchmark a given function.
    """
    start_time = time.time()
    result = func(*args, **kwargs)
    elapsed_time = time.time() - start_time
    return result, elapsed_time


def median_composite_builtin(ds):
    """
    Compute the median composite along the 'time' dimension using Xarray's built-in median.
    """
    return ds.median("time", skipna=True, keep_attrs=True)


def median_composite_ufunc(ds):
    """
    Compute the median composite along the 'time' dimension using NumPy's nanmedian and Xarray ufunc.
    """
    composite = xr.apply_ufunc(
        np.nanmedian,
        ds,
        input_core_dims=[["time"]],
        output_core_dims=[[]],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[np.float32],
    )
    return composite


def quantile_composite_builtin(ds, quantile):
    """
    Compute the quantile composite along the 'time' dimension using Xarray's built-in quantile.
    """
    return ds.quantile(quantile, dim="time", skipna=True, keep_attrs=True)


def quantile_composite_ufunc(ds, quantile):
    """
    Compute the quantile composite along the 'time' dimension using NumPy's nanpercentile and Xarray ufunc.
    """
    composite = xr.apply_ufunc(
        np.nanpercentile,
        ds,
        kwargs={"q": quantile * 100},
        input_core_dims=[["time"]],
        output_core_dims=[[]],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[np.float32],
    )
    return composite


def compute_percentage_difference(array1, array2):
    """
    Compute the percentage difference between two arrays.
    """
    diff = np.abs(array1 - array2)
    return np.mean(diff / (np.abs(array1) + np.abs(array2) + 1e-10)) * 100


def main():
    # Simulated dataset for testing
    time = pd.date_range("2023-01-01", "2023-01-10")
    y = np.linspace(-10, 10, 50)
    x = np.linspace(-10, 10, 50)
    data = np.random.rand(len(time), len(y), len(x))

    ds = xr.DataArray(
        data,
        coords={"time": time, "y": y, "x": x},
        dims=("time", "y", "x"),
        name="test_data",
    )

    # Quantile to use for quantile composite tests
    quantile = 0.15

    # Benchmark each method
    methods = [
        ("Median (Xarray Built-in)", median_composite_builtin, {"ds": ds}),
        ("Median (Xarray Ufunc)", median_composite_ufunc, {"ds": ds}),
        (
            "Quantile (Xarray Built-in)",
            quantile_composite_builtin,
            {"ds": ds, "quantile": quantile},
        ),
        (
            "Quantile (Xarray Ufunc)",
            quantile_composite_ufunc,
            {"ds": ds, "quantile": quantile},
        ),
    ]

    results = []
    outputs = {}

    for method_name, method, kwargs in methods:
        result, elapsed_time = benchmark(method, **kwargs)
        outputs[method_name] = result
        results.append({"Method": method_name, "Time (s)": elapsed_time})

    # Compare results and calculate percentage differences
    comparisons = [
        ("Median (Xarray Built-in)", "Median (Xarray Ufunc)"),
        ("Quantile (Xarray Built-in)", "Quantile (Xarray Ufunc)"),
    ]

    for method1, method2 in comparisons:
        diff = compute_percentage_difference(
            outputs[method1].values, outputs[method2].values
        )
        results.append(
            {
                "Method": f"Comparison {method1} vs {method2}",
                "Time (s)": f"Diff: {diff:.6f}%",
            }
        )

    # Display results
    results_df = pd.DataFrame(results)
    print(results_df)


if __name__ == "__main__":
    main()

In [ ]:
composite

In [ ]:
xx.median("time")

In [ ]:
import numpy as np
import xarray as xr


def percentile_composite(ds, q, nodata=np.nan):
    """
    Create a percentile composite from a datacube.

    Args:
        ds (xr.Dataset): Input Dataset with dimensions (time, y, x).
        q (float): Percentile to compute (e.g., 15 for 15th percentile).
        nodata (float, optional): Value representing nodata. Default is NaN.

    Returns:
        xr.Dataset: Composite with dimensions (y, x).
    """
    # Mask nodata values
    ds_masked = ds.where(ds != nodata)

    # Apply percentile calculation
    def nanpercentile(data, axis, q):
        return np.nanpercentile(data, q=q, axis=axis)

    composite = xr.apply_ufunc(
        nanpercentile,
        ds_masked,
        input_core_dims=[["time"]],  # Operate along the "time" dimension
        output_core_dims=[[]],  # Output has no "time" dimension
        kwargs={"q": q},  # Percentile value
        dask="parallelized",  # Enable Dask support
        output_dtypes=[float],  # Output type
        keep_attrs=True,  # Retain attributes
    )

    # Preserve nodata value in output
    for var in composite.data_vars:
        composite[var].attrs["nodata"] = nodata

    return composite


percentile_composite(xx, 15)

In [ ]:
import numpy as np
import xarray as xr

# Sample dataset for testing
xx = xr.Dataset(
    {
        "nir": (["time", "y", "x"], np.random.random((10, 100, 100))),
        "green": (["time", "y", "x"], np.random.random((10, 100, 100))),
    },
    coords={
        "time": np.arange(10),
        "y": np.linspace(0, 100, 100),
        "x": np.linspace(0, 100, 100),
    },
)


def median_composite(ds):
    """
    Compute a median composite over the 'time' dimension.

    Args:
        ds (xr.Dataset): Input Dataset with dimensions (time, y, x).

    Returns:
        xr.Dataset: Composite with dimensions (y, x).
    """
    # Apply median calculation using xr.apply_ufunc
    composite = xr.apply_ufunc(
        np.nanmedian,  # Use numpy's nanmedian
        ds,  # Input dataset
        input_core_dims=[["time"]],  # Operate along the "time" dimension
        output_core_dims=[[]],  # Result has no "time" dimension
        dask="parallelized",  # Support for Dask arrays
        output_dtypes=[ds["nir"].dtype],  # Output dtype inferred from input
        keep_attrs=True,  # Retain attributes from the input dataset
    )
    return composite


# Test the function
composite = median_composite(xx)
print(composite)

In [ ]:
xr.apply_ufunc(
    np.nanpercentile,  # Handle NaNs during computation
    xx,  # Dataset to process
    input_core_dims=[["time"]],  # Specify "time" as the axis for computation
    output_core_dims=[["y", "x"]],  # Output should retain "y" and "x" dimensions
    kwargs={"q": 15},  # Pass the percentile to compute
    # dask="parallelized",  # Enable Dask support
    # output_dtypes=[float],  # Define the output data type
)

In [ ]:
import numpy as np
import xarray as xr


def composite(ds: xr.Dataset, percentile: float, nodata: float = np.nan) -> xr.Dataset:
    """
    Create a percentile composite from a datacube.

    Args:
        ds (xr.Dataset): The input datacube with dimensions ("time", "y", "x").
        percentile (float): The percentile to compute (e.g., 15 for 15th percentile).
        nodata (float): The value representing nodata. Default is NaN.

    Returns:
        xr.Dataset: The composite with the specified percentile.
    """
    # 1. Mask nodata values
    ds_masked = ds.where(ds != nodata)

    # 2. Apply percentile calculation
    composite = xr.apply_ufunc(
        np.nanpercentile,  # Handle NaNs during computation
        ds_masked,  # Dataset to process
        input_core_dims=[["time"]],  # Specify "time" as the axis for computation
        output_core_dims=[["y", "x"]],  # Output should retain "y" and "x" dimensions
        kwargs={"q": percentile},  # Pass the percentile to compute
        dask="parallelized",  # Enable Dask support
        output_dtypes=[float],  # Define the output data type
    )

    # 3. Preserve nodata values
    for var in ds.data_vars:
        composite[var] = composite[var].where(~composite[var].isnull(), nodata)
        composite[var].attrs["nodata"] = nodata
        composite[var] = composite[var].rio.write_nodata(nodata)

    # 4. Return the composite
    return composite


composite(s2, 15)

In [ ]:
import numpy as np

agg = s2.reduce(np.percentile, dim="time", q=0.15)

In [ ]:
r.nir.plot()

In [ ]:
s2.isel(time=10).blue.plot()

In [ ]:
s2.nir.hvplot(x="x", y="y", geo=True)

In [ ]:
s2.nirhvplot(x="x", y="y")

In [ ]:
cop_dem["data"]

In [ ]:
cop_dem = cop_dem.compute()

In [ ]:
deltadtm